# 自定义层

构造一个没有任何参数的自定义层

In [1]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        '''
        X.mean()计算输入张量X的所有元素的平均值
        X - X.mean()将每个元素减去这个均值，实现数据中心化
        结果是一个均值为0的新张量
        '''
        return X - X.mean()

layer = CenteredLayer() # 实例
layer(torch.FloatTensor([1, 2, 3, 4, 5])) # 浮点张量

tensor([-2., -1.,  0.,  1.,  2.])

将层作为组件合并到更复杂的模型中

In [2]:
net = nn.Sequential(nn.Linear(8, 128), # 全连接层，将 8维 输入映射为 128维 输出
                    CenteredLayer()) # 自定义的中心化层，会减去输入数据的均值

Y = net(torch.rand(4, 8)) # 生成形状为(4, 8)的随机张量（4个样本，每个8维特征）
Y.mean() # 计算最终输出张量Y所有元素的均值

tensor(-4.6566e-09, grad_fn=<MeanBackward0>)

带参数的层

In [3]:
class MyLinear(nn.Module):
    # in_units: 输入特征维度（5）；units: 输出特征维度（3）
    def __init__(self, in_units, units):
        super().__init__() # 初始化父类
        # nn.Parameter:模型参数，会被自动跟踪和训练
        self.weight = nn.Parameter(torch.randn(in_units, units)) # 形状为(5, 3)的权重矩阵，随机初始化
        self.bias = nn.Parameter(torch.randn(units,)) # 形状为 (3,) 的偏置向量，随机初始化
    def forward(self, X):
        # 矩阵乘法，计算线性变换；加上偏置
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        # 应用ReLU激活函数（负值归零）
        return F.relu(linear) 

linear = MyLinear(5, 3)
# 访问该层的权重参数
linear.weight

Parameter containing:
tensor([[-0.4303,  0.4544, -1.8435],
        [ 0.3427,  0.7560, -0.3311],
        [-0.9465,  1.3326, -0.0471],
        [ 1.6853, -1.2154, -0.8468],
        [ 0.0887, -0.6610,  0.2225]], requires_grad=True)

使用自定义层直接执行前向传播计算

In [4]:
linear(torch.rand(2, 5)) # 生成一个形状为(2, 5)的随机张量

tensor([[0.4390, 0.9349, 0.0000],
        [0.0000, 2.0294, 0.2914]])

使用自定义层构建模型

In [5]:
'''
输入：torch.rand(2,64)→形状(2,64)，数值范围[0,1)
第一层：MyLinear(64,8)
线性变换：matmul((2,64),(64, 8))→(2,8)，加偏置
ReLU激活：负数变0→形状仍为(2, 8)，所有元素≥0
第二层：MyLinear(8,1)
线性变换：matmul((2,8),(8,1))→(2,1) ，加偏置
ReLU激活：负数变0→最终形状(2, 1)，所有元素≥0
'''
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[21.0786],
        [14.1205]])